In [ ]:
from PIL import Image
import pytesseract
import pandas as pd
import os
import re
# Configura la ruta de Tesseract si es necesario (solo para Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\paola\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

In [5]:
# Función para extraer texto de una imagen
def extract_text_from_image(image_path):
    # Abre la imagen
    img = Image.open(image_path)
    # Usa pytesseract para extraer el texto
    text = pytesseract.image_to_string(img, lang='spa')  # 'spa' para español
    return text

In [ ]:

# Función para extraer coordenadas UTM del texto
def extract_utm_coordinates(text, image_name):
    # Busca el patrón de coordenadas UTM en el texto
    lines = text.split('\n')
    for line in lines:
        if '13Q' in line:  # Busca la zona UTM (13Q en este caso)
            parts = line.split()
            if len(parts) >= 3:
                try:
                    easting = float(parts[-2])  # Coordenada Este
                    northing = float(parts[-1])  # Coordenada Norte
                    return easting, northing
                except ValueError:
                    print(f"Error al convertir coordenadas en {image_name}: {line}")
            else:
                print(f"Formato inesperado en {image_name}: {line}")
    print(f"No se encontraron coordenadas en {image_name}.")
    return None, None

In [20]:
# Función para extraer coordenadas UTM usando expresiones regulares
def extract_utm_coordinates(text, image_name):
    # Definir el patrón de búsqueda
    pattern = r'13Q\s+(\d{6})\s+(\d{7})'  # Busca "13Q" seguido de 6 y 7 dígitos
    match = re.search(pattern, text)
    
    if match:
        easting = float(match.group(1))  # Primer grupo de dígitos (Easting)
        northing = float(match.group(2))  # Segundo grupo de dígitos (Northing)
        return easting, northing
    else:
        print(f"No se encontraron coordenadas en {image_name}.")
        return None, None

In [7]:
# Carpeta con las fotos
folder_path = r'D:\BlueMX\Sitios ARR\Fotos'
files = os.listdir(folder_path)

# Lista para almacenar los datos
data = []

In [21]:
# Procesar cada imagen
for i, file in enumerate(files):
    if file.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(folder_path, file)
        print(f"Procesando {file}...")
        
        # Extraer texto de la imagen
        text = extract_text_from_image(image_path)
        print(f"Texto extraído de {file}:\n{text}\n")
        
        # Extraer coordenadas UTM del texto
        easting, northing = extract_utm_coordinates(text, file)
        if easting is not None and northing is not None:
            data.append({
                'Punto': i + 1,
                'Archivo': file,
                'Este (Easting)': easting,
                'Norte (Northing)': northing
            })

Procesando P1.jpg...
Texto extraído de P1.jpg:
- - - 28 ene. 2025 9:16:13 a. m.
13Q 440255 2460433
— BLUEMX



Procesando P10.jpg...
Texto extraído de P10.jpg:


No se encontraron coordenadas en P10.jpg.
Procesando P100.jpg...
Texto extraído de P100.jpg:


No se encontraron coordenadas en P100.jpg.
Procesando P101.jpg...
Texto extraído de P101.jpg:


No se encontraron coordenadas en P101.jpg.
Procesando P102.jpg...
Texto extraído de P102.jpg:


No se encontraron coordenadas en P102.jpg.
Procesando P103.jpg...
Texto extraído de P103.jpg:
()



No se encontraron coordenadas en P103.jpg.
Procesando P104.jpg...
Texto extraído de P104.jpg:
321 NW



No se encontraron coordenadas en P104.jpg.
Procesando P11.jpg...
Texto extraído de P11.jpg:


No se encontraron coordenadas en P11.jpg.
Procesando P12.jpg...
Texto extraído de P12.jpg:
= . X ene2025 171:14:04
- 7| 13048023 245808y

F

7

NE N M'zá!']án—'—xTeñfg?
— x - Nayaritl


No se encontraron coordenadas en P12.jpg.
Procesando P13.jpg...
Tex

In [23]:
# Crear un DataFrame y guardar como CSV
df = pd.DataFrame(data)
df.to_csv(r'D:\BlueMX\Sitios ARR\Puntos\sitios_arr.csv', index=False)

print(f"Archivo CSV creado exitosamente. Se procesaron {len(data)} de {len(files)} imágenes.")

Archivo CSV creado exitosamente. Se procesaron 63 de 104 imágenes.
